In [1]:
# Benodigde packages

import json            # Package om .json files in te laden (bvb kolomnamen zijn zo opgeslagen)
import getpass         # Package om een paswoordveldje te genereren.
import mysql.connector # MySQL package
import numpy as np
import os
import pandas as pd    # Populaire package voor data-verwerking
import sys
from basics import*

sys.version_info       # Check python versie, wij veronderstellen 3.6

sys.version_info(major=3, minor=6, micro=4, releaselevel='final', serial=0)

In [2]:
def verbind_met_GB(username, hostname, gegevensbanknaam):
    """
    Maak verbinding met een externe gegevensbank
    
    :param  username:          username van de gebruiker, string
    :param  hostname:          naam van de host, string.
                               Dit is in het geval van een lokale server gewoon 'localhost'
    :param  gegevensbanknaam:  naam van de gegevensbank, string.
    :return connection:        connection object, dit is wat teruggeven wordt 
                               door connect() methods van packages die voldoen aan de DB-API
    """
    
    password = getpass.getpass() # Genereer vakje voor wachtwoord in te geven
    
    connection = mysql.connector.connect(host=hostname,
                                         user=username,
                                         passwd=password,
                                         db=gegevensbanknaam)
    return connection


def run_query(connection, query):
    """
    Voer een query uit op een reeds gemaakte connectie, geeft het resultaat van de query terug
    """
    
    # Making a cursor and executing the query
    cursor = connection.cursor()
    cursor.execute(query)
    
    # Collecting the result and casting it in a pd.DataFrame
    res = cursor.fetchall()
    
    return res


def res_to_df(query_result, column_names):
    """
    Giet het resultaat van een uitgevoerde query in een 'pandas dataframe'
    met vooraf gespecifieerde kolomnamen.
    
    Let op: Het resultaat van de query moet dus exact evenveel kolommen bevatten
    als kolomnamen die je meegeeft. Als dit niet het geval is, is dit een indicatie
    dat je oplossing fout is. (Gezien wij de kolomnamen van de oplossing al cadeau doen)
    
    """
    df = pd.DataFrame(query_result, columns=column_names)
    return df

In [3]:
# Dictionary van kolomnamen inladen

filename = os.path.join(os.getcwd(), 'solution', 'all_q_colnam.json')
col_names = json.load(open(filename, 'r'))    

col_names # Inspecteer dictionary

{'query_01': ['tname', 'year', 'HomeRun'],
 'query_02': ['nameFirst', 'nameLast', 'birthYear', 'birthMonth', 'birthDay'],
 'query_03': ['nameFirst', 'nameLast', 'tname'],
 'query_04': ['tname', 'rank', 'W', 'L', 'nameFirst', 'nameLast'],
 'query_05': ['tname'],
 'query_06': ['tname', 'yearID', 'rank', 'W', 'L'],
 'query_07': ['nameLast', 'nameFirst'],
 'query_08': ['birthState',
  'avg_weight',
  'avg_height',
  'avg_HomeRun',
  'avg_Saves'],
 'query_09': ['yearID', 'tname', 'HomeRun'],
 'query_10': ['yearID', 'tname', 'rank', 'Games']}

In [4]:
def query_01(connection, column_names):
    # Bouw je query
    query="""
    SELECT t.name, t.yearID, t.HR
    FROM   Teams as t
    ORDER by t.HR DESC;
    """
    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [5]:
username = 'root'      # Vervang dit als je via een andere user queries stuurt
hostname = 'localhost' # Als je een databank lokaal draait, is dit localhost.
db = 'lahman2016'      # Naam van de gegevensbank op je XAMPP Mysql server

# We verbinden met de gegevensbank
c = verbind_met_GB(username, hostname, db)

········


In [6]:
# De voorbeeldquery heeft dezelfde kolomnamen als query 1, dus we gebruiken die
kolommen = col_names['query_01'] 

# Functie uitvoeren, geeft resultaat van de query in een DataFrame
df = query_01(c, col_names['query_01'])

# We inspecteren de eerste paar resultaten (voor alles te zien: laat .head() weg)
df.head() 

,tname,year,HomeRun
0,Seattle Mariners,1997,264
1,Texas Rangers,2005,260
2,Baltimore Orioles,1996,257
3,Toronto Blue Jays,2010,257
4,Baltimore Orioles,2016,253


In [28]:
def query_02(connection, column_names, datum = '1980-01-16'):
    # Bouw je query
    query="""
    SELECT Master.nameFirst, Master.nameLast, 
           Master.birthYear, Master.birthMonth, Master.birthDay
    FROM   Master
    WHERE  Master.debut> {}
    ORDER by Master.nameLast;
    """.format(datum)
    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [29]:
username = 'root'      # Vervang dit als je via een andere user queries stuurt
hostname = 'localhost' # Als je een databank lokaal draait, is dit localhost.
db = 'lahman2016'      # Naam van de gegevensbank op je XAMPP Mysql server

# We verbinden met de gegevensbank
c = verbind_met_GB(username, hostname, db)

········


In [30]:
# De voorbeeldquery heeft dezelfde kolomnamen als query 1, dus we gebruiken die
kolommen = col_names['query_02'] 

# Functie uitvoeren, geeft resultaat van de query in een DataFrame
df = query_02(c, col_names['query_02'], datum = '1980-01-16')

# We inspecteren de eerste paar resultaten (voor alles te zien: laat .head() weg)
df.head() 

,nameFirst,nameLast,birthYear,birthMonth,birthDay
0,David,Aardsma,1981,12,27
1,Don,Aase,1954,9,8
2,Andy,Abad,1972,8,25
3,Fernando,Abad,1985,12,17
4,Glenn,Abbott,1951,2,16


In [10]:
def query_03(connection, column_names):
    # Bouw je query
    query="""
    SELECT DISTINCT MS.nameFirst, MS.nameLast, t.name
    FROM   Teams as t, Managers as MG, Master as MS
    WHERE  MG.teamID = t.teamID AND MG.plyrMgr = 'N'
           AND MS.playerID = MG.playerID
    ORDER by t.name;
    """
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [11]:
username = 'root'      # Vervang dit als je via een andere user queries stuurt
hostname = 'localhost' # Als je een databank lokaal draait, is dit localhost.
db = 'lahman2016'      # Naam van de gegevensbank op je XAMPP Mysql server

# We verbinden met de gegevensbank
c = verbind_met_GB(username, hostname, db)

········


In [12]:
# De voorbeeldquery heeft dezelfde kolomnamen als query 1, dus we gebruiken die
kolommen = col_names['query_03'] 

# Functie uitvoeren, geeft resultaat van de query in een DataFrame
df = query_03(c, col_names['query_03'])

# We inspecteren de eerste paar resultaten (voor alles te zien: laat .head() weg)
df.head()

,nameFirst,nameLast,tname
0,Ed,Curtis,Altoona Mountain City
1,Mike,Scioscia,Anaheim Angels
2,Terry,Collins,Anaheim Angels
3,Joe,Maddon,Anaheim Angels
4,Chip,Hale,Arizona Diamondbacks


In [37]:
def query_04(connection, column_names, datum_x='1980-01-01', datum_y='1980-01-01'):
    # Bouw je query
    query="""
    SELECT DISTINCT t.name, t.Rank, t.W, t.L, MS.nameFirst, MS.nameLast
    FROM   Teams as t, Managers as MG, Master as MS, Halloffame as HF
    WHERE  MS.playerID = MG.playerID AND HF.playerID = MG.playerID
           AND MG.teamID = t.teamID
           AND HF.yearid > {}
           AND MG.yearID < {}
    ORDER by t.name, t.Rank;
    """.format(datum_x, datum_y)
    ####hoe variablen binnen krijgen 
    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [38]:
username = 'root'      # Vervang dit als je via een andere user queries stuurt
hostname = 'localhost' # Als je een databank lokaal draait, is dit localhost.
db = 'lahman2016'      # Naam van de gegevensbank op je XAMPP Mysql server

# We verbinden met de gegevensbank
c = verbind_met_GB(username, hostname, db)

········


In [39]:
# De voorbeeldquery heeft dezelfde kolomnamen als query 1, dus we gebruiken die
kolommen = col_names['query_04'] 

# Functie uitvoeren, geeft resultaat van de query in een DataFrame
df = query_04(c, col_names['query_04'], datum_x='1980-01-01', datum_y='1980-01-01')

# We inspecteren de eerste paar resultaten (voor alles te zien: laat .head() weg)
df


,tname,rank,W,L,nameFirst,nameLast
0,Atlanta Braves,1,101,59,Bobby,Cox
1,Atlanta Braves,1,90,72,Bobby,Cox
2,Atlanta Braves,1,89,73,Bobby,Cox
3,Atlanta Braves,1,93,69,Bobby,Cox
4,Atlanta Braves,1,94,68,Bobby,Cox
5,Atlanta Braves,1,98,64,Bobby,Cox
6,Atlanta Braves,1,104,58,Bobby,Cox
7,Atlanta Braves,1,90,54,Bobby,Cox
8,Atlanta Braves,1,96,66,Bobby,Cox
9,Atlanta Braves,1,101,61,Bobby,Cox


In [50]:
def query_05(connection, column_names):
    # Bouw je query
    query="""
    SELECT DISTINCT t.name
    FROM   Teams as t
    WHERE  (SELECT  COUNT(*)
            FROM    Teams as t, Managers as MG
            WHERE   t.yearID>'1980-01-01'
                    AND MG.teamID = T.teamID AND MG.plyrMgr = 'Y')>=1;
    """
    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [51]:
username = 'root'      # Vervang dit als je via een andere user queries stuurt
hostname = 'localhost' # Als je een databank lokaal draait, is dit localhost.
db = 'lahman2016'      # Naam van de gegevensbank op je XAMPP Mysql server

# We verbinden met de gegevensbank
c = verbind_met_GB(username, hostname, db)

········


In [52]:
# De voorbeeldquery heeft dezelfde kolomnamen als query 1, dus we gebruiken die
kolommen = col_names['query_05'] 

# Functie uitvoeren, geeft resultaat van de query in een DataFrame
df = query_05(c, col_names['query_05'])

# We inspecteren de eerste paar resultaten (voor alles te zien: laat .head() weg)
df

,tname
0,Boston Red Stockings
1,Chicago White Stockings
2,Cleveland Forest Citys
3,Fort Wayne Kekiongas
4,New York Mutuals
5,Philadelphia Athletics
6,Rockford Forest Citys
7,Troy Haymakers
8,Washington Olympics
9,Baltimore Canaries


In [25]:
def query_06(connection, column_names, salaris=200000):
    # Bouw je query
    query="""
    SELECT t.name, t.YearID, t.Rank, t.W, t.L
    FROM   Teams as t, Salaries as S
    GROUP BY t.teamID
    HAVING {} < ALL(SELECT S.salary
            FROM Teams as t, Salaries as S
            WHERE t.yearID = S.yearID AND t.teamID = S.teamID);
    """.format(salaris)
    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [26]:
username = 'root'      # Vervang dit als je via een andere user queries stuurt
hostname = 'localhost' # Als je een databank lokaal draait, is dit localhost.
db = 'lahman2016'      # Naam van de gegevensbank op je XAMPP Mysql server

# We verbinden met de gegevensbank
c = verbind_met_GB(username, hostname, db)

········


In [27]:
# De voorbeeldquery heeft dezelfde kolomnamen als query 1, dus we gebruiken die
kolommen = col_names['query_06'] 

# Functie uitvoeren, geeft resultaat van de query in een DataFrame
df = query_06(c, col_names['query_06'], salaris=200000)

# We inspecteren de eerste paar resultaten (voor alles te zien: laat .head() weg)
df

,tname,yearID,rank,W,L


In [40]:
def query_07(connection, column_names):
    # Bouw je query
    query="""
    SELECT DISTINCT MS.nameLast, MS.nameFirst
    FROM  Managers as MG, Master as MS, Awardsmanagers as AM
    
    WHERE MG.playerID = MS.playerID AND MS.playerID = AM.playerID
    GROUP BY AM.playerID
    HAVING COUNT(DISTINCT AM.awardID)=2

    ORDER by MS.nameLast;
    """
    #    AND   AM.awardID LIKE '%SN%'  
    # AND   AM.awardID LIKE '%BW%' 

    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [41]:
username = 'root'      # Vervang dit als je via een andere user queries stuurt
hostname = 'localhost' # Als je een databank lokaal draait, is dit localhost.
db = 'lahman2016'      # Naam van de gegevensbank op je XAMPP Mysql server

# We verbinden met de gegevensbank
c = verbind_met_GB(username, hostname, db)

········


In [43]:
# De voorbeeldquery heeft dezelfde kolomnamen als query 1, dus we gebruiken die
kolommen = col_names['query_07'] 

# Functie uitvoeren, geeft resultaat van de query in een DataFrame
df = query_07(c, col_names['query_07'])

# We inspecteren de eerste paar resultaten (voor alles te zien: laat .head() weg)
df


,nameLast,nameFirst
0,Alou,Felipe
1,Anderson,Sparky
2,Baker,Dusty
3,Baylor,Don
4,Black,Buddy
5,Bochy,Bruce
6,Bowa,Larry
7,Cox,Bobby
8,Frey,Jim
9,Gardenhire,Ron


In [ ]:
def query_08(connection, column_names, jaar=1990, lengte=75):
    # Bouw je query
    query="""
    SELECT   MS.birthState, AVG(MS.weight), AVG(MS.height),
             AVG(B.HR), AVG(P.SV)
    FROM     Master as MS, Batting as B, Pitching as P
    WHERE    AVG(SELECT MS.Height
                 FROM   Master as MS, Halloffame as HF
                 WHERE  HF.yearID >= {} AND
                        MS.playerID = HF.playerID) > {}
    GROUP BY MS.birthState
    ORDER BY MS.birthState;
    """.format(jaar,lengte)
    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [ ]:
username = 'root'      # Vervang dit als je via een andere user queries stuurt
hostname = 'localhost' # Als je een databank lokaal draait, is dit localhost.
db = 'lahman2016'      # Naam van de gegevensbank op je XAMPP Mysql server

# We verbinden met de gegevensbank
c = verbind_met_GB(username, hostname, db)

In [ ]:
# De voorbeeldquery heeft dezelfde kolomnamen als query 1, dus we gebruiken die
kolommen = col_names['query_08'] 

# Functie uitvoeren, geeft resultaat van de query in een DataFrame
df = query_08(c, col_names['query_08'], jaar=1990, lengte=75)

# We inspecteren de eerste paar resultaten (voor alles te zien: laat .head() weg)
df

In [ ]:
def query_09(connection, column_names, jaar=1975):
    # Bouw je query
    query="""
    SELECT t.Yearid, t.name, t.HR
    FROM Team as t
    WHERE t.YearID = {} AND t.HR = MIN(t.HR)
    IN (SELECT TOP 2 *
    FROM Teams as t
    ORDER BY t.HR)
    """.format(jaar)
    
    # Stap 2 & 3
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [ ]:
username = 'root'      # Vervang dit als je via een andere user queries stuurt
hostname = 'localhost' # Als je een databank lokaal draait, is dit localhost.
db = 'lahman2016'      # Naam van de gegevensbank op je XAMPP Mysql server

# We verbinden met de gegevensbank
c = verbind_met_GB(username, hostname, db)

In [ ]:
# De voorbeeldquery heeft dezelfde kolomnamen als query 1, dus we gebruiken die
kolommen = col_names['query_09'] 

# Functie uitvoeren, geeft resultaat van de query in een DataFrame
df = query_09(c, col_names['query_09'])

# We inspecteren de eerste paar resultaten (voor alles te zien: laat .head() weg)
df

In [3]:
def query_10(connection, column_names, jaar=1990):
    # Bouw je query
    
    query="""
    SELECT t.yearID, t.name, t.Rank, t.G
    FROM   Teams as t, Appearances as A
    WHERE  t.teamID = A.teamID AND t.yearID={}
           AND (SELECT  COUNT(DISTINCT AP.playerID)
                FROM    Appearances as A, Awardsplayers as AP, Teams as t
                WHERE   A.teamID = t.teamID AND A.playerID = AP.playerID AND T.yearID= {}
                        AND (SELECT COUNT(*)
                             FROM   Appearances as A, Awardsplayers as AP, Teams as t
                             WHERE  A.teamID = t.teamID AND t.yearID= {}
                             AND    AP.playerID in (SELECT  A.playerID 
                                                     FROM   Appearances as A, Teams as t
                                                     WHERE  A.teamID = t.TeamID)                 
                       )=1)>0;
    ORDER by t.name, t.Rank
    """.format(jaar,jaar,jaar)
    res = run_query(connection, query)         # Query uitvoeren
    df = res_to_df(res, column_names)          # Query in DataFrame brengen
    
    return df

In [ ]:
username = 'root'      # Vervang dit als je via een andere user queries stuurt
hostname = 'localhost' # Als je een databank lokaal draait, is dit localhost.
db = 'lahman2016'      # Naam van de gegevensbank op je XAMPP Mysql server

# We verbinden met de gegevensbank
c = verbind_met_GB(username, hostname, db)

In [ ]:
# De voorbeeldquery heeft dezelfde kolomnamen als query 1, dus we gebruiken die
kolommen = col_names['query_10'] 

# Functie uitvoeren, geeft resultaat van de query in een DataFrame
df = query_10(c, col_names['query_10'], jaar=1990)

# We inspecteren de eerste paar resultaten (voor alles te zien: laat .head() weg)
df.head() 